In [1]:
OUTPUT_PATH = '../../data/'

In [2]:
%run '../../schema_definition/schema.py'

In [3]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [4]:
from pyspark.shell import spark
from pyspark.sql import functions as F

21/10/16 04:14:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.8.9 (default, Aug 21 2021 15:53:23)
Spark context Web UI available at http://localhost:4042
Spark context available as 'sc' (master = local[*], app id = local-1634318074587).
SparkSession available as 'spark'.


## List post codes based on fastest response
***
- Fastest response hint ( Refer columns Request date and implementation Date )

In [66]:
gold_df = spark.read.parquet(OUTPUT_PATH + "gold")
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT DISTINCT
                        PostCode, Response
                        FROM fastest_response
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)
drop_date_column  = fastest_response_df.drop('date')
drop_date_column.coalesce(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'fastest_response', mode='overwrite')
df = drop_date_column.toPandas()
df

,PostCode,Response
0,2081,-1
1,2586,-1
2,2582,-1
3,2722,-1
4,2587,-1
...,...,...
147,2582,-368
148,2582,-378
149,2582,-424
150,2582,-1020


## Response
***

- When the number of events reach 1000, output the events to a JSON file.

In [67]:
gold_df = spark.read.parquet(OUTPUT_PATH + "gold")
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("response")

fastest_response_query = """
                        SELECT DISTINCT
                        *
                        FROM response
                        """
fastest_response_df = spark.sql(fastest_response_query)
drop_date_column  = fastest_response_df.drop('date')
drop_date_column.coalesce(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'response', mode='overwrite')
df = drop_date_column.toPandas()
df

,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode,HashKey,Response
0,10041,543,2017-09-07,1.0,B2C,ELE,NUOS,2017-08-29,PAID,7,12.78,v1_0,307508,EN122192582-N-12,2017-09-07 11:16:00,RESI,2587,3f0a010875ce3aafff08c3c9e95b9cdf3751961e4293be...,-9
1,10026,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-15,PAID,0,4.28,v1_0,307509,EN121701122-N-3,2017-09-07 12:02:00,RESI,2587,60bf724767cdbcfb141684f2924ecd684af61eab92746d...,-23
2,10081,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-24,PAID,6,31.04,v1_0,307508,EN122071667-N-12,2017-09-07 11:13:00,RESI,2587,aa67901dec4f1424fc2bc2cc3a8eb6bfa07561bf9ad075...,-14
3,10043,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,1,7.27,v1_0,307312,EA983_158001245365-N-2,2017-09-11 09:25:00,BSME,2581,f9f78dead2bf82664a3cca5bce3085ae74b8cb6fb43762...,-1
4,10005,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,7,0.65,v1_0,307312,EA983_90003707292-N-1,2017-09-11 09:27:00,RESI,2581,63f163ec77531fabb50bdbb6b64827c75a949ed6aa8fe7...,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29636,10033,108,2017-09-08,1.0,B2B,ELE,NUOS,2017-09-07,PAID,8,7.96,v1_0,307561,EN63320785-N-4,2017-09-13 09:06:00,RESI,2581,8535b1d72260f90175823d960b2c992b1d9951818637f3...,-1
29637,10049,1164,2017-09-11,1.0,B2B,ELE,NUOS,2017-08-03,PAID,6,0.00,v1_0,307676,1.14E+11,2017-09-14 11:12:00,RESI,2581,1b31bf56e39f1f98d266afaaaec2d20b84d347fc2eb40d...,-39
29638,10021,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-30,PAID,5,2.77,v1_0,307510,EN122266899-N-5,2017-09-07 12:54:00,RESI,2587,56a962ba319b7caff04cb8dac24f0f9d6f39d3dfe8fc7a...,-8
29639,10003,543,2017-09-07,1.0,B2B,ELE,NUOS,2017-08-25,PAID,4,3.51,v1_0,307510,EN122117977-N-11,2017-09-07 12:39:00,RESI,2587,9b0bf7a250f30d5fd79c6de1aa12ed2aa6d45ca84929e7...,-13
